# Indeed Job Scraper

The objective of this program is to scrape/ extract a batch of job postings from Indeed. The program currently allows the user to input multiple queries and locations into the scraper, but otherwise leaves the other settings at their defaults. After specifying the settings, the scraper returns the 
* title
* location
* company
* salary
* synopsis

The output will be shown in a preview below, and stored in a local csv file. The local csv file will be used for the main project, analyzing the job postings and be able to gain insight from each posting's synopsis.

Link to project: https://github.com/mhuh22/Thinkful/blob/master/Bootcamp/Unit%207/Analysis%20on%20Indeed%20Data%20Scientist%20Postings.ipynb

In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import time
import os

In [2]:
def parse(query,metro, n):
    
    # Target url, and n number of pages to scrape
    url = "https://www.indeed.com/jobs?q={}&l={}&start={}"
    
    # Create dataframe that we'll return later
    df = pd.DataFrame(columns=["Title","Location","Company","Salary", "Synopsis", "Query", "Metro"])
    
    # Scrape first n pages
    for i in range(0,n):
        html = requests.get(url.format(query, metro, i))
        time.sleep(1)
        soup = BeautifulSoup(html.content, 'html.parser', from_encoding="utf-8")
        
        for each in soup.find_all(class_= "result" ):
            try: 
                title = each.find(class_='jobtitle').text.replace('\n', '')
            except:
                title = 'None'
            try:
                location = each.find('span', {'class':"location" }).text.replace('\n', '')
            except:    
                try:
                    location = each.find('div', {'class':"location" }).text.replace('\n', '')
                except:
                    location = 'None'
            try: 
                company = each.find(class_='company').text.replace('\n', '')
            except:
                company = 'None'
            try:
                salary = each.find('span', {'class':'no-wrap'}).text
            except:
                salary = 'None'
            # Can also extract url (not sure if it will be used, though)
#             try:
#                 link = 'https://www.indeed.com/viewjob' + each.find('a', href=True)['href'][7:]
#             except:    
#                 link = 'None'
            synopsis = each.find('span', {'class':'summary'}).text.replace('\n', '')
            df = df.append({'Title':title, 'Location':location, 'Company':company, 'Salary':salary, 'Synopsis':synopsis, 'Query': query, 'Metro': metro}, ignore_index=True)
    return df

In [ ]:
# Lists of positions and locations to search
positions = ['data+analyst', 'data+scientist', 'data+engineer']
cities = ['united states']
n = 100

# Create the dataframe
data = pd.DataFrame()

# Scrape all positions in all cities specified, and append to dataframe
for position in positions:
    for city in cities:
        data = pd.concat([data, parse(position, city, n)])

# Drop duplicate posts (typically caused by sponsored positions reappearing)
data = data.drop_duplicates()
        
# Preview the data
data.head(10)

In [ ]:
# Return size of dataframe
data.shape

In [ ]:
# Save scraped data to a local directory
if os.path.exists("data/job_board.csv"):
    os.remove("data/job_board.csv")
data.to_csv('data/job_board.csv', encoding='utf-8')